In [1]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- 1. Pfad-Setup (wie in ET.ipynb) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import load_target, load_ifo_features
# --- MODELL-IMPORT ---
from src.models.EN import ForecastModel  # Elastic Net

print('PROJECT_ROOT =', PROJECT_ROOT)

# --- 2. Modellname & Outputs ---
MODEL_NAME = "elastic_net"  # analog zu ET
outputs_for_model(MODEL_NAME)
print(f'Modell {MODEL_NAME} wird getunt.')

# --- 3. Daten laden (Gleis 1/2) ---
y = load_target()          # ΔIP mit DatetimeIndex
X_ifo = load_ifo_features()
idx = y.index.intersection(X_ifo.index)
y, X_ifo = y.loc[idx], X_ifo.loc[idx]
print('Daten geladen. Shapes:', X_ifo.shape, y.shape)

# --- 4. Base-Config (für schnellen Test: debug) ---
def base_cfg_fast() -> GlobalConfig:
    cfg = GlobalConfig(preset="debug")  # kleine Splits, schnell
    # Policy-Parameter (können so bleiben)
    cfg.policy_window   = 12
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 2
    return cfg

cfg0 = base_cfg_fast()

# --- 5. Minimales FE/HP-Grid (schnell!) ---
# FE-Parameter stark reduziert (nur expanding corr, 1 Lag-Logik, kleine K1)
corr_spec = dict(DEFAULT_CORR_SPEC)   # expanding window corr aus config

lag_candidates_list       = [(1, 3, 6)]     # klein
top_k_lags_list           = [1]
use_rm3_list              = [True]
k1_topk_list              = [100]           # kleine Vorauswahl
redundancy_param_list     = [0.90]          # moderat
dr_options_list           = [{'dr_method': 'none'}]  # keine PCA/PLS
target_block_options      = [None]          # keine Target-only Blocks (Setup I)
weighting_options         = [{"sample_weight_decay": None}]

# Elastic Net HPs minimal:
# alpha (Mixing l1_ratio), lambda (Penalty)
alpha_list   = [0.5]       # 0=Ridge, 1=Lasso; 0.5 = EN
lambda_list  = [1e-2]      # schwache Regularisierung
seed_list    = [42]

def build_model_grid_fast():
    hp_grid = []
    for lags in lag_candidates_list:
        for k_lags in top_k_lags_list:
            for rm3 in use_rm3_list:
                for k1 in k1_topk_list:
                    for red in redundancy_param_list:
                        for dr_opt in dr_options_list:
                            for block_set in target_block_options:
                                for weight_hp in weighting_options:
                                    for a in alpha_list:
                                        for lam in lambda_list:
                                            for sd in seed_list:
                                                hp = {
                                                    # FE
                                                    'lag_candidates': lags,
                                                    'top_k_lags_per_feature': k_lags,
                                                    'use_rm3': rm3,
                                                    'k1_topk': k1,
                                                    'redundancy_param': red,
                                                    **dr_opt,
                                                    'corr_tag': 'expanding',
                                                    'corr_spec': corr_spec,
                                                    'target_block_set': block_set,
                                                    **weight_hp,
                                                    # EN
                                                    'alpha': a,     # -> l1_ratio
                                                    'lambda': lam,  # -> sklearn alpha
                                                    'seed': sd,
                                                }
                                                hp_grid.append(hp)
    return hp_grid

model_grid = build_model_grid_fast()
print("HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid leer")

# --- 6. Stage A/B Lauf ---
if not model_grid:
    raise RuntimeError("Leeres Grid – nichts zu tun.")

# KLEINE Stages für Speed
shortlist = run_stageA(
    model_name=MODEL_NAME,
    model_ctor=lambda hp: ForecastModel(hp),
    model_grid=model_grid,
    X=X_ifo,
    y=y,
    cfg=cfg0,
    keep_top_k_final=min(1, len(model_grid)),     # nur 1 Kandidat behalten
    min_survivors_per_block=1                     # aggressives Halving
)

run_stageB(
    model_name=MODEL_NAME,
    model_ctor=lambda hp: ForecastModel(hp),
    shortlist=shortlist,
    X=X_ifo,
    y=y,
    cfg=cfg0
)

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} für Ergebnisse.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell elastic_net wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Daten geladen. Shapes: (407, 2160) (407,)
HP-Kombinationen: 1
Erstes HP-Set: {'lag_candidates': (1, 3, 6), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'target_block_set': None, 'sample_weight_decay': None, 'alpha': 0.5, 'lambda': 0.01, 'seed': 42}
[Stage A] Using FULL FE pipeline (Gleis 1 & 2).
[Stage A][Block 1] train_end=48, OOS=49-60 | configs=1
  - Config 1/1
    · Month 5/12 processed | running...RMSE=3.4599
    · Month 10/12 processed | running...RMSE=3.8793
    · Month 12/12 processed | running...RMSE=4.0896
[Stage A][Block 1] kept 1 configs (floor=1).
[Stage A][Block 2] train_end=60, OOS=61-72 | configs=1
  - Config 1/1


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.574e-01, tolerance: 8.101e-03
  model = cd_fast.enet_coordinate_descent(


    · Month 5/12 processed | running...RMSE=11.8606
    · Month 10/12 processed | running...RMSE=12.5516
    · Month 12/12 processed | running...RMSE=12.5974
[Stage A][Block 2] kept 1 configs (floor=1).
[Stage A] Shortlist saved with 1 configs.
[Stage B] Using FULL FE pipeline (Gleis 1 & 2).
[Stage B] Month origin t=73 | evaluating 1 configs | active=1


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.342e+00, tolerance: 9.867e-03
  model = cd_fast.enet_coordinate_descent(


[Stage B] Month origin t=74 | evaluating 1 configs | active=1


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.839e+00, tolerance: 1.443e-01
  model = cd_fast.enet_coordinate_descent(


[Stage B] Month origin t=405 | evaluating 1 configs | active=1


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.329e+00, tolerance: 1.444e-01
  model = cd_fast.enet_coordinate_descent(


[Stage B] done → /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/stageB/elastic_net/monthly
[Stage B] summary.csv & summary_active.txt geschrieben.

Done. Check outputs/stageA|stageB/elastic_net für Ergebnisse.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.275e-01, tolerance: 1.450e-01
  model = cd_fast.enet_coordinate_descent(
